In [4]:
import os, glob, numpy as np
import subprocess
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import pandas as pd
import mlflow
import mlflow.pytorch

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# MLFlow setup
mlflow.set_tracking_uri("http://192.5.86.161:8000/")
mlflow.set_experiment("PannsMLP")
try: 
    mlflow.end_run() # end pre-existing run, if there was one
except:
    pass
finally:
    mlflow.start_run(log_system_metrics=True) # Start MLFlow run
gpu_info = next(
    (subprocess.run(cmd, capture_output=True, text=True).stdout for cmd in ["nvidia-smi", "rocm-smi"] 
     if subprocess.run(f"command -v {cmd}", shell=True, capture_output=True).returncode == 0),
    "No GPU found."
)
mlflow.log_text(gpu_info, "gpu-info.txt")

from mlflow.system_metrics.system_metrics_logger import SystemMetricsLogger

logger = SystemMetricsLogger()
print("System metrics logger initialized:", logger._should_log())

2025/04/30 19:00:16 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/04/30 19:00:16 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2025/04/30 19:00:16 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Using device: cuda
🏃 View run spiffy-sow-547 at: http://192.5.86.161:8000/#/experiments/1/runs/50f9a3318bf44705ae22e29e2143f9db
🧪 View experiment at: http://192.5.86.161:8000/#/experiments/1


ModuleNotFoundError: No module named 'mlflow.system_metrics.system_metrics_logger'

In [2]:
EMB_DIR = '/home/jovyan/Features/embeddings'

meta = pd.read_csv('/home/jovyan/Data/birdclef-2025/train.csv')
label2idx = {lab:i for i, lab in enumerate(sorted(meta['primary_label'].unique()))}
num_classes = len(label2idx)

# 1) Gather embedding files
all_paths  = sorted(glob.glob(os.path.join(EMB_DIR, '**', '*_emb.npz'), recursive=True))
all_labels = [int(np.load(p)['label']) for p in all_paths]

# 2) Split into train/test, try stratify then fallback
try:
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_paths, all_labels,
        test_size=0.2,
        random_state=42
    )
except ValueError:
    print("Warning: stratify failed (too few samples in some classes), splitting without stratify.")
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_paths, all_labels,
        test_size=0.2,
        shuffle=True,
        random_state=42
    )

# 3) Dataset definition
class EmbeddingDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):
        data = np.load(self.paths[idx])
        emb  = data['embedding'].astype(np.float32)
        lbl  = int(data['label'])
        return torch.from_numpy(emb), torch.tensor(lbl)

# 4) Instantiate & wrap in DataLoaders
train_ds = EmbeddingDataset(train_paths)
test_ds  = EmbeddingDataset(test_paths)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

print(f"Train samples: {len(train_ds)}, Test samples: {len(test_ds)}")

Train samples: 8988, Test samples: 2248


In [3]:
# Cell 3 — Define MLP, criterion, optimizer & count params
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout=0.5):
        super().__init__()
        layers = []
        dims = [input_dim] + hidden_dims
        for i in range(len(hidden_dims)):
            layers += [
                nn.Linear(dims[i], dims[i+1]),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout)
            ]
        layers.append(nn.Linear(dims[-1], num_classes))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

sample_emb, sample_lbl = next(iter(train_loader))
input_dim   = sample_emb.shape[1]

model     = MLPClassifier(input_dim, [1024, 512], num_classes, dropout=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params:,}")


Total trainable parameters: 2,728,654


In [4]:
# Log hyperparameters
mlflow.log_params({
    "input_dim": input_dim,
    "hidden_dims": [1024, 512],
    "dropout": 0.5,
    "batch_size": 32,
    "lr": 1e-3,
    "num_epochs": 20
})

In [5]:
# Training loop with MLFlow logging
num_epochs = 20
best_acc   = 0.0
checkpoint_path = 'best_panns_mlp_checkpoint.pt'

for epoch in range(1, num_epochs+1):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total = 0
    for emb, lbl in tqdm(train_loader, desc=f"Epoch {epoch} ▶ Train"):
        emb, lbl = emb.to(device), lbl.to(device)
        optimizer.zero_grad()
        logits = model(emb)
        loss   = criterion(logits, lbl)
        loss.backward()
        optimizer.step()

        train_loss    += loss.item() * emb.size(0)
        train_correct += (logits.argmax(dim=1) == lbl).sum().item()
        total         += emb.size(0)

    train_loss /= total
    train_acc   = train_correct / total

    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for emb, lbl in tqdm(test_loader, desc=f"Epoch {epoch} ✅ Val"):
            emb, lbl = emb.to(device), lbl.to(device)
            logits = model(emb)
            loss   = criterion(logits, lbl)

            val_loss    += loss.item() * emb.size(0)
            val_correct += (logits.argmax(dim=1) == lbl).sum().item()
            val_total   += emb.size(0)

    val_loss /= val_total
    val_acc   = val_correct / val_total

    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    }, step=epoch)

    print(f"\nEpoch {epoch:02d} | Train: loss={train_loss:.4f}, acc={train_acc:.4f} | Val: loss={val_loss:.4f}, acc={val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict':       model.state_dict(),
            'optimizer_state_dict':   optimizer.state_dict(),
            'best_validation_acc':    best_acc,
            'input_dim':              input_dim,
            'hidden_dims':            [1024, 512],
            'num_classes':            num_classes
        }, checkpoint_path)
        print(f"✔️  New best model saved (epoch {epoch}, val_acc={val_acc:.4f})")
        mlflow.pytorch.log_model(model, "best_panns_mlp")

mlflow.log_metric("best_val_accuracy", best_acc)
mlflow.end_run()

print(f"\n🎉 Best validation accuracy: {best_acc:.4f}")

Epoch 1 ▶ Train:   0%|          | 0/281 [00:00<?, ?it/s]

Epoch 1 ✅ Val:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 01 | Train: loss=4.5005, acc=0.0765 | Val: loss=4.0391, acc=0.1361
✔️  New best model saved (epoch 1, val_acc=0.1361)


2025/04/30 18:49:43 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/30 18:49:49 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/30 18:49:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 2 ▶ Train:   0%|          | 0/281 [00:00<?, ?it/s]

...

2025/04/30 18:52:39 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/30 18:52:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/30 18:52:39 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under mlflow-artifacts:/1/3a16f8b54e0c4b58b5eca4db2e13708f/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.


Epoch 20 ▶ Train:   0%|          | 0/281 [00:00<?, ?it/s]

Epoch 20 ✅ Val:   0%|          | 0/71 [00:00<?, ?it/s]

2025/04/30 18:52:44 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/04/30 18:52:44 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!



Epoch 20 | Train: loss=2.1322, acc=0.4557 | Val: loss=2.2550, acc=0.4795
🏃 View run masked-toad-913 at: http://192.5.86.161:8000/#/experiments/1/runs/3a16f8b54e0c4b58b5eca4db2e13708f
🧪 View experiment at: http://192.5.86.161:8000/#/experiments/1

🎉 Best validation accuracy: 0.4795


In [32]:
# Cell 5 — (Optional) Load best checkpoint for inference or continued training
ckpt = torch.load('best_panns_mlp_checkpoint.pt', map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
optimizer.load_state_dict(ckpt['optimizer_state_dict'])
print(f"Loaded checkpoint from epoch {ckpt['epoch']} with val_acc={ckpt['best_validation_acc']:.4f}")


Loaded checkpoint from epoch 13 with val_acc=0.5565


/tmp/ipykernel_8068/2758929225.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('best_panns_mlp_checkpoint.pt', map_location=device)
